In [41]:
import sys
import json
import nltk
from tree_sitter import Language, Parser
from collections import Counter
PY_LANGUAGE = Language('D:/python/final/jsonl/train\python_train_0.jsonl/build/my-languages.so', 'python')
parser = Parser()
parser.set_language(PY_LANGUAGE)
json_file=[]
with open("./train/python_train_0.jsonl/python_train_0.jsonl") as rf:
    for line in rf.readlines():
        dic = json.loads(line)
        dic_ = {}
        dic_["docstrings"] = dic["docstring"]
        dic_["code_tokens"] = dic["code_tokens"]
        dic_["code"] = dic["code"]
        dic_["docstring_tokens"] = dic["docstring_tokens"]
        json_file.append(dic_)


def DFS_Tree(node):
    if not node:
        return []
    stack = [node]
    res = []
    while stack:
        node = stack.pop()
        res.append(node.type)
        for child in node.children:
            stack.append(child)
            res.append(child.type)
    return res


def BFS_Tree(node):
    if node == None:
        return []
    node_list = []
    nodes = []
    node_list.append(node)
    nodes.append(node.type)
    while node_list:
        node = node_list.pop(0)
        for child in node.children:
            node_list.append(child)
            nodes.append(child.type)
    return nodes

def load_data(json_file):
    pl = []
    nl = []
    num_examples = 0
    idx = 0
    for program in json_file:
#         program["docstrings"] = program["docstrings"].strip()
#         idx +=1
#         print(idx)
        nl.append(["BOS"] + nltk.word_tokenize(program["docstrings"].lower()) + ["EOS"])
        # split chinese sentence into characters
        pl.append(["BOS"] + DFS_Tree(program["AST"].root_node) + ["EOS"])
    return nl, pl


In [42]:
for idx in range(len(json_file)):

    docstrings = []
    code = json_file[idx]["code"]
    for token in json_file[idx]["code_tokens"]:
        if len(token) and token[0] == '#':
            docstrings.append(token)
            json_file[idx]["code_tokens"].remove(token)
    json_file[idx]["AST"] = parser.parse(bytes(code, "utf8"))
    json_file[idx]["docstrings"] = json_file[idx]["docstring_tokens"] + docstrings
    nl = ' '.join(json_file[idx]["docstrings"])
    json_file[idx]["docstrings"] = nl
print(json_file[0]["docstrings"])
# tree = parser.parse(bytes(code, "utf8"))

nl , pl = load_data(json_file)
print(nl[0])
print(pl[0])

Trains a k - nearest neighbors classifier for face recognition . # Loop through each person in the training set # Loop through each training image for the current person # If there are no people (or too many people) in a training image, skip the image. # Add face encoding for current image to the training set # Determine how many neighbors to use for weighting in the KNN classifier # Create and train the KNN classifier # Save the trained KNN classifier
['BOS', 'trains', 'a', 'k', '-', 'nearest', 'neighbors', 'classifier', 'for', 'face', 'recognition', '.', '#', 'loop', 'through', 'each', 'person', 'in', 'the', 'training', 'set', '#', 'loop', 'through', 'each', 'training', 'image', 'for', 'the', 'current', 'person', '#', 'if', 'there', 'are', 'no', 'people', '(', 'or', 'too', 'many', 'people', ')', 'in', 'a', 'training', 'image', ',', 'skip', 'the', 'image', '.', '#', 'add', 'face', 'encoding', 'for', 'current', 'image', 'to', 'the', 'training', 'set', '#', 'determine', 'how', 'many', '

In [43]:
UNK_IDX = 1
PAD_IDX = 0
def build_dict(sentences, max_words=50000):
    word_count = Counter()
    for sentence in sentences:
        for s in sentence:
            word_count[s] += 1
    ls = word_count.most_common(max_words)
    total_words = len(ls) + 2
    word_dict = {w[0]: index+2 for index, w in enumerate(ls)}
    word_dict["UNK"] = UNK_IDX
    word_dict["PAD"] = PAD_IDX
    return word_dict, total_words

nl_dict, nl_total_words = build_dict(nl)
pl_dict, pl_total_words = build_dict(pl)
print(pl_dict, pl_total_words)
inv_en_dict = {v: k for k, v in en_dict.items()}
inv_cn_dict = {v: k for k, v in cn_dict.items()}

{'identifier': 2, '"': 3, '(': 4, ')': 5, '.': 6, 'attribute': 7, ',': 8, 'call': 9, 'argument_list': 10, '=': 11, 'expression_statement': 12, 'string': 13, ':': 14, 'assignment': 15, 'block': 16, '[': 17, ']': 18, 'subscript': 19, 'if': 20, 'if_statement': 21, 'keyword_argument': 22, 'integer': 23, 'comment': 24, 'comparison_operator': 25, 'return_statement': 26, 'return': 27, 'none': 28, 'function_definition': 29, 'def': 30, 'parameters': 31, 'default_parameter': 32, 'in': 33, 'binary_operator': 34, 'not': 35, 'pair': 36, 'for': 37, 'else': 38, 'list': 39, 'BOS': 40, 'module': 41, 'EOS': 42, 'else_clause': 43, '{': 44, '}': 45, 'dictionary': 46, 'false': 47, '==': 48, 'for_statement': 49, 'boolean_operator': 50, 'not_operator': 51, 'is': 52, 'true': 53, '+': 54, 'escape_sequence': 55, 'tuple': 56, 'raise_statement': 57, 'raise': 58, '-': 59, 'pattern_list': 60, 'and': 61, '**': 62, 'parenthesized_expression': 63, 'except_clause': 64, 'except': 65, 'elif_clause': 66, 'elif': 67, '*': 

NameError: name 'en_dict' is not defined

In [44]:
def encode(nl_sentences, pl_sentences, nl_dict, pl_dict, sort_by_len=False):
    '''
        Encode the sequences. 
    '''
    length = len(nl_sentences)
    out_nl_sentences = [[nl_dict.get(w, 0) for w in sent] for sent in nl_sentences]
    out_pl_sentences = [[pl_dict.get(w, 0) for w in sent] for sent in pl_sentences]

    # sort sentences by english lengths
    def len_argsort(seq):
        return sorted(range(len(seq)), key=lambda x: len(seq[x]))
       
    # 把中文和英文按照同样的顺序排序
    if sort_by_len:
        sorted_index = len_argsort(out_nl_sentences)
        out_nl_sentences = [out_nl_sentences[i] for i in sorted_index]
        out_pl_sentences = [out_pl_sentences[i] for i in sorted_index]
        
    return out_nl_sentences, out_pl_sentences

train_nl, train_pl = encode(nl, pl, nl_dict, pl_dict)
# print(train_nl[0])
inv_pl_dict = {v: k for k, v in pl_dict.items()}
inv_nl_dict = {v: k for k, v in nl_dict.items()}
# k = 0
# print(" ".join([inv_nl_dict[i] for i in train_nl[k]]))
# print(" ".join([inv_pl_dict[i] for i in train_pl[k]]))

[5, 3689, 7, 611, 20, 1501, 1770, 1448, 14, 2346, 3499, 2, 4, 452, 348, 73, 6407, 17, 3, 186, 45, 4, 452, 348, 73, 186, 127, 14, 3, 90, 6407, 4, 18, 128, 38, 79, 2977, 15, 30, 709, 925, 2977, 16, 17, 7, 186, 127, 11, 459, 3, 127, 2, 4, 59, 2346, 553, 14, 90, 127, 8, 3, 186, 45, 4, 468, 538, 925, 1770, 8, 58, 14, 4759, 17, 3, 4760, 1448, 4, 53, 12, 492, 3, 4760, 1448, 4, 215, 3, 788, 4760, 1448, 6]
BOS trains a k - nearest neighbors classifier for face recognition . # loop through each person in the training set # loop through each training image for the current person # if there are no people ( or too many people ) in a training image , skip the image . # add face encoding for current image to the training set # determine how many neighbors to use for weighting in the knn classifier # create and train the knn classifier # save the trained knn classifier EOS
BOS module function_definition function_definition def identifier parameters : block block expression_statement expression_stateme

In [ ]:
def get_minibatches(n, minibatch_size, shuffle=True):
    idx_list = np.arange(0, n, minibatch_size) # [0, 1, ..., n-1]
    if shuffle:
        np.random.shuffle(idx_list)
    minibatches = []
    for idx in idx_list:
        minibatches.append(np.arange(idx, min(idx + minibatch_size, n)))
    return minibatches

def prepare_data(seqs):
    lengths = [len(seq) for seq in seqs]
    n_samples = len(seqs)
    max_len = np.max(lengths)

    x = np.zeros((n_samples, max_len)).astype('int32')
    x_lengths = np.array(lengths).astype("int32")
    for idx, seq in enumerate(seqs):
        x[idx, :lengths[idx]] = seq
    return x, x_lengths #x_mask

def gen_examples(nl_sentences, pl_sentences, batch_size):
    minibatches = get_minibatches(len(nl_sentences), batch_size)
    all_ex = []
    for minibatch in minibatches:
        mb_nl_sentences = [nl_sentences[t] for t in minibatch]
        mb_pl_sentences = [pl_sentences[t] for t in minibatch]
        mb_x, mb_x_len = prepare_data(mb_nl_sentences)
        mb_y, mb_y_len = prepare_data(mb_pl_sentences)
        all_ex.append((mb_x, mb_x_len, mb_y, mb_y_len))
    return all_ex

batch_size = 64
train_data = gen_examples(train_nl, train_pl, batch_size)
random.shuffle(train_data)


In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, enc_hidden_size, dec_hidden_size, dropout=0.2):
        super(Encoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, enc_hidden_size, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(enc_hidden_size * 2, dec_hidden_size)

    def forward(self, x, lengths):
        sorted_len, sorted_idx = lengths.sort(0, descending=True)
        x_sorted = x[sorted_idx.long()]
        embedded = self.dropout(self.embed(x_sorted))
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, sorted_len.long().cpu().data.numpy(), batch_first=True)
        packed_out, hid = self.rnn(packed_embedded)
        out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
        _, original_idx = sorted_idx.sort(0, descending=False)
        out = out[original_idx.long()].contiguous()
        hid = hid[:, original_idx.long()].contiguous()
        
        hid = torch.cat([hid[-2], hid[-1]], dim=1)
        hid = torch.tanh(self.fc(hid)).unsqueeze(0)

        return out, hid

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hidden_size, dec_hidden_size):
        super(Attention, self).__init__()

        self.enc_hidden_size = enc_hidden_size
        self.dec_hidden_size = dec_hidden_size

        self.linear_in = nn.Linear(enc_hidden_size*2, dec_hidden_size, bias=False)
        self.linear_out = nn.Linear(enc_hidden_size*2 + dec_hidden_size, dec_hidden_size)
        
    def forward(self, output, context, mask):
        # output: batch_size, output_len, dec_hidden_size
        # context: batch_size, context_len, 2*enc_hidden_size
    
        batch_size = output.size(0)
        output_len = output.size(1)
        input_len = context.size(1)
        
        context_in = self.linear_in(context.view(batch_size*input_len, -1)).view(                
            batch_size, input_len, -1) # batch_size, context_len, dec_hidden_size
        
        # context_in.transpose(1,2): batch_size, dec_hidden_size, context_len 
        # output: batch_size, output_len, dec_hidden_size
        attn = torch.bmm(output, context_in.transpose(1,2)) 
        # batch_size, output_len, context_len

        attn.data.masked_fill(mask, -1e6)

        attn = F.softmax(attn, dim=2) 
        # batch_size, output_len, context_len

        context = torch.bmm(attn, context) 
        # batch_size, output_len, enc_hidden_size
        
        output = torch.cat((context, output), dim=2) # batch_size, output_len, hidden_size*2

        output = output.view(batch_size*output_len, -1)
        output = torch.tanh(self.linear_out(output))
        output = output.view(batch_size, output_len, -1)
        return output, attn

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, enc_hidden_size, dec_hidden_size, dropout=0.2):
        super(Decoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.attention = Attention(enc_hidden_size, dec_hidden_size)
        self.rnn = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.out = nn.Linear(dec_hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def create_mask(self, x_len, y_len):
        # a mask of shape x_len * y_len
        device = x_len.device
        max_x_len = x_len.max()
        max_y_len = y_len.max()
        x_mask = torch.arange(max_x_len, device=x_len.device)[None, :] < x_len[:, None]
        y_mask = torch.arange(max_y_len, device=x_len.device)[None, :] < y_len[:, None]
        mask = (1 - x_mask[:, :, None] * y_mask[:, None, :]).byte()
        return mask
    
    def forward(self, ctx, ctx_lengths, y, y_lengths, hid):
        sorted_len, sorted_idx = y_lengths.sort(0, descending=True)
        y_sorted = y[sorted_idx.long()]
        hid = hid[:, sorted_idx.long()]
        
        y_sorted = self.dropout(self.embed(y_sorted)) # batch_size, output_length, embed_size

        packed_seq = nn.utils.rnn.pack_padded_sequence(y_sorted, sorted_len.long().cpu().data.numpy(), batch_first=True)
        out, hid = self.rnn(packed_seq, hid)
        unpacked, _ = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        _, original_idx = sorted_idx.sort(0, descending=False)
        output_seq = unpacked[original_idx.long()].contiguous()
        hid = hid[:, original_idx.long()].contiguous()

        mask = self.create_mask(y_lengths, ctx_lengths)

        output, attn = self.attention(output_seq, ctx, mask)
        output = F.log_softmax(self.out(output), -1)
        
        return output, hid, attn

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, x, x_lengths, y, y_lengths):
        encoder_out, hid = self.encoder(x, x_lengths)
        output, hid, attn = self.decoder(ctx=encoder_out, 
                    ctx_lengths=x_lengths,
                    y=y,
                    y_lengths=y_lengths,
                    hid=hid)
        return output, attn
    
    def translate(self, x, x_lengths, y, max_length=100):
        encoder_out, hid = self.encoder(x, x_lengths)
        preds = []
        batch_size = x.shape[0]
        attns = []
        for i in range(max_length):
            output, hid, attn = self.decoder(ctx=encoder_out, 
                    ctx_lengths=x_lengths,
                    y=y,
                    y_lengths=torch.ones(batch_size).long().to(y.device),
                    hid=hid)
            y = output.max(2)[1].view(batch_size, 1)
            preds.append(y)
            attns.append(attn)
        return torch.cat(preds, 1), torch.cat(attns, 1)

In [ ]:
dropout = 0.2
embed_size = hidden_size = 100
encoder = Encoder(vocab_size=en_total_words,
                       embed_size=embed_size,
                      enc_hidden_size=hidden_size,
                       dec_hidden_size=hidden_size,
                      dropout=dropout)
decoder = Decoder(vocab_size=cn_total_words,
                      embed_size=embed_size,
                      enc_hidden_size=hidden_size,
                       dec_hidden_size=hidden_size,
                      dropout=dropout)
model = Seq2Seq(encoder, decoder)
model = model.to(device)
loss_fn = LanguageModelCriterion().to(device)
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
train(model, train_data, num_epochs=30)